# Code to analyse output BAHA5P, before (first fit) and after adjustig to max stable gain

28 February 2021 Guido Cattani, revision 5 November 2022

Is it worth to try to set gain to maximal stable values?

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import wilcoxon as wilcoxon
from scipy.stats import shapiro as shapiro
from scipy.stats import ttest_rel as ttest_rel

In [2]:
# read output force levels measured with a 65 dB ISDS input signal
def read_BCD_output_65():
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/Headband/BCD_band_output.xlsx'
    p_in = Path(f_in)   
    df = pd.read_excel(p_in, header=0, nrows=85)
    df = df.drop(['Unnamed: 0'], axis=1)
    df.set_index('Study_ID', inplace=True)
    df = df.fillna(pd.NA)
    return df

In [3]:
def select_bh5(df):
    # select BAHA5P data
    is_baha5p =  df['Device']=='BAHA5P'
    df_baha5p = df[is_baha5p]
    df_baha5p.pop('Device')
    return(df_baha5p)

In [4]:
# read maximal output in force levels measured with a 65 dB ISDS input signal
def read_BAHA5P_maximal_output_65():
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/Headband/BAHA5P_band_maximal_output_65.xlsx'
    p_in = Path(f_in)   
    df = pd.read_excel(p_in, header=0, nrows=35)
    df = df.drop(['Unnamed: 0', 'Device'], axis=1)
    df.set_index('Study_ID', inplace=True)
    df = df.fillna(pd.NA)
    return df

In [5]:
def diff_output():
    bcd_fit = read_BCD_output_65()
    bh5_fit = select_bh5(bcd_fit)
    bh5_max = read_BAHA5P_maximal_output_65()
    diff = bh5_max - bh5_fit
    diff = diff.dropna()
    return diff

In [6]:
# read max stable output BAHA5P 
datamax = read_BAHA5P_maximal_output_65()

In [7]:
# read output BAHA5P 
data = read_BCD_output_65()
data = select_bh5(data)
idx = datamax.index
data = data.loc[idx]

In [8]:
# adjust column names, change format columns labels 'f_Hz' to f

clmns = datamax.columns
l = list()
for clm in clmns:
    l.append(clm)


In [9]:
# check normality output group BAHA5P & BP110 with Shapiro-Wilk test

shp = dict()

for i in range(0, 16):
    a = data.iloc[:, i]
    b = datamax.iloc[:, i]
    f = l[i]
    shapiro_stat, pVal = shapiro(a) # output bh5 scipy.stats Shapiro-Wilk test test
    shapiro_stat_max, pVal_max = shapiro(b) # max output bh5 scipy.stats Shapiro-Wilk test test
    out_is_normal = False if pVal < 0.05 else True
    max_is_normal = False if pVal_max < 0.05 else True
    both_is_normal = True if out_is_normal and max_is_normal else False
    st = {f: [shapiro_stat, pVal, out_is_normal, shapiro_stat_max, pVal_max, max_is_normal, both_is_normal]}
    shp.update(st)

shapiro_test = pd.DataFrame.from_dict(shp)
dish =  {0: 'Shapiro test statistic output BH5', 1: 'p-value output BH5', 
         2: 'normally distributed in BH5', 3: 'Shapiro test statistic max output BH5', 
         4: 'p-value output max BH5', 5: 'normally distributed in max BH5', 6: 'both normally distributed'}
shapiro_test = shapiro_test.rename(index=dish)
shapiro_test

,250_Hz,315_Hz,400_Hz,500_Hz,630_Hz,800_Hz,1000_Hz,1250_Hz,1600_Hz,2000_Hz,2500_Hz,3150_Hz,4000_Hz,5000_Hz,6300_Hz,8000_Hz
Shapiro test statistic output BH5,0.91359,0.96033,0.96031,0.925318,0.910732,0.908027,0.948305,0.970115,0.970424,0.955695,0.886964,0.920576,0.899335,0.902128,0.938126,0.951402
p-value output BH5,0.00936,0.233926,0.233607,0.02035,0.007782,0.006545,0.100514,0.446424,0.455018,0.169378,0.001797,0.014813,0.003795,0.004513,0.049078,0.125132
normally distributed in BH5,False,True,True,False,False,False,True,True,True,True,False,False,False,False,False,True
Shapiro test statistic max output BH5,0.929858,0.970616,0.961967,0.974788,0.975304,0.980787,0.984768,0.973023,0.936652,0.882692,0.860396,0.847708,0.846256,0.856761,0.915714,0.938434
p-value output max BH5,0.027704,0.460387,0.261728,0.587152,0.603883,0.784476,0.898338,0.531459,0.044285,0.001399,0.000401,0.000205,0.00019,0.00033,0.010749,0.050148
normally distributed in max BH5,False,True,True,True,True,True,True,True,False,False,False,False,False,False,False,True
both normally distributed,False,True,True,False,False,False,True,True,False,False,False,False,False,False,False,True


In [10]:
# Make a filter based on both normally distibuted criterium (Shapiro test)
bool_filter = shapiro_test.loc['both normally distributed']
bool_filter = pd.concat([bool_filter] * 3, axis=1).T
bool_filter = bool_filter.reset_index(drop=True)

In [11]:
''' Compare output of BAHA5P at 65 dB, first fit versus maximal stable
This is a test for the null hypothesis that two related or repeated samples have identical 
average (expected) values.'''

ttst = dict()

for i in range(0, 16):
    a = data.iloc[:, i]
    b = datamax.iloc[:, i]
    f = l[i]
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html
    statistic, pVal = ttest_rel(a, b)
    statistic = round(statistic, 4)
    pVal = round(pVal, 4)
    pVal_pro=pVal*100
    st = {f: [statistic, pVal, pVal_pro]}
    ttst.update(st)
t_test_paired = pd.DataFrame.from_dict(ttst, dtype='float')

# apply boolean filter from Shapiro test
t_test_paired = t_test_paired[bool_filter]
t_test_paired

# present results
idx = {0 : 'T-test paired', 1 : 'p value', 2 : 'p value %'}
t_test_paired.rename(index=idx, inplace=True)
t_test_paired

,250_Hz,315_Hz,400_Hz,500_Hz,630_Hz,800_Hz,1000_Hz,1250_Hz,1600_Hz,2000_Hz,2500_Hz,3150_Hz,4000_Hz,5000_Hz,6300_Hz,8000_Hz
T-test paired,NaN,-2.8719,-2.5637,NaN,NaN,NaN,-4.2593,-5.3606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7479
p value,NaN,0.0070,0.0149,NaN,NaN,NaN,0.0002,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4597
p value %,NaN,0.7000,1.4900,NaN,NaN,NaN,0.0200,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.9700


In [12]:
diff = diff_output()

In [13]:
len(diff)

35

In [14]:
diff

,250_Hz,315_Hz,400_Hz,500_Hz,630_Hz,800_Hz,1000_Hz,1250_Hz,1600_Hz,2000_Hz,2500_Hz,3150_Hz,4000_Hz,5000_Hz,6300_Hz,8000_Hz
Study_ID,,,,,,,,,,,,,,,,
21,-1.2,-3.1,1.3,1.3,0.9,0.8,0.7,1.0,1.6,2.4,1.3,0.7,0.8,1.2,1.5,1.2
48,3.1,2.7,1.9,2.2,2.2,2.4,2.6,2.1,1.3,0.7,0.1,-0.1,0.1,1.2,2.2,2.8
52,4.7,9.2,7.4,6.2,6.3,5.7,4.2,1.6,0.2,-0.2,-0.4,-0.5,-0.7,-0.3,-0.1,0.3
54,-0.3,-2.8,-3.5,-1.6,-1.2,-0.5,1.6,6.6,8.9,7.2,3.0,1.1,-0.1,-0.7,-0.9,-0.7
55,0.7,1.5,0.0,-0.9,-0.6,-0.4,-0.3,0.1,1.9,3.9,2.4,0.6,-0.8,-1.2,-1.0,-0.8
56,4.7,5.1,4.0,3.2,3.4,3.7,3.5,3.4,2.1,0.7,-0.2,-0.5,-1.4,-1.9,-1.7,-1.6
57,0.0,0.2,0.2,-0.1,-0.2,-0.1,0.0,-0.2,0.0,0.4,0.2,0.0,-0.1,-0.2,-0.1,-0.1
58,0.0,1.1,1.5,0.9,0.8,0.6,0.5,0.5,1.5,3.2,2.3,1.0,0.5,0.2,-0.2,-0.4
59,-0.7,-1.1,0.1,0.4,0.5,0.6,0.8,2.9,4.0,3.8,1.3,0.1,0.2,0.3,0.3,0.3


In [15]:
# calculate quantiles of diff. between max stable output and ouput before optimisation for group BAHA5
quantiles = [0.10, 0.50, 0.90]
quanti = diff.quantile(q=quantiles)
quanti = quanti.reset_index(drop=True)
diq = {0:'Diff. Max stable P10', 1:'Diff. Max stable P50', 2:'Diff. Max stable P90'} 
quanti.rename(index=diq, inplace=True)
quanti

,250_Hz,315_Hz,400_Hz,500_Hz,630_Hz,800_Hz,1000_Hz,1250_Hz,1600_Hz,2000_Hz,2500_Hz,3150_Hz,4000_Hz,5000_Hz,6300_Hz,8000_Hz
Diff. Max stable P10,-0.22,-0.52,-1.08,-0.74,-0.48,-0.36,-0.30,-0.20,0.14,0.14,-0.10,-0.36,-0.86,-1.12,-1.18,-1.26
Diff. Max stable P50,0.20,0.70,0.40,0.40,0.40,0.40,0.70,1.30,1.90,2.90,1.70,0.60,0.10,0.00,-0.20,-0.20
Diff. Max stable P90,3.46,3.32,3.18,3.12,3.40,3.76,3.46,3.94,5.58,6.96,5.62,3.82,1.04,0.86,1.06,1.10


In [16]:
# make list with column names (frequencies)

clmns = diff.columns
l = list()
for clm in clmns:
    l.append(clm)

In [17]:
''' Compare output of BAHA5P at 65 dB, first fit versus maximal stable
The Wilcoxon signed-rank test tests the null hypothesis that 
two related paired samples come from the same distribution. 
In particular, it tests whether the distribution of the differences x - y is symmetric 
about zero. It is a non-parametric version of the paired T-test.'''

wlcx = dict()

for i in range(0, 16):
    x = diff.iloc[:, i]
    f = l[i]
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html
    statistic, pVal = wilcoxon(x)
    statistic = round(statistic, 4)
    pVal = round(pVal, 4)
    pVal_promil=pVal*100
    st = {f: [statistic, pVal, pVal_promil]}
    wlcx.update(st)
wilcoxon_test = pd.DataFrame.from_dict(wlcx, dtype='float')
idx = {0 : 'Wilcoxon signed rank statistic', 1 : 'p value', 2 : 'p value %'}
wilcoxon_test.rename(index=idx, inplace=True)
wilcoxon_test

,250_Hz,315_Hz,400_Hz,500_Hz,630_Hz,800_Hz,1000_Hz,1250_Hz,1600_Hz,2000_Hz,2500_Hz,3150_Hz,4000_Hz,5000_Hz,6300_Hz,8000_Hz
Wilcoxon signed rank statistic,47.5000,107.0000,131.5000,148.0000,97.0000,107.0000,67.0000,39.0,6.5,9.0,25.0,38.0,237.5000,239.5000,214.0000,239.500
p value,0.0011,0.0033,0.0077,0.0105,0.0031,0.0019,0.0001,0.0,0.0,0.0,0.0,0.0,0.4421,0.4633,0.1531,0.216
p value %,0.1100,0.3300,0.7700,1.0500,0.3100,0.1900,0.0100,0.0,0.0,0.0,0.0,0.0,44.2100,46.3300,15.3100,21.600


In [18]:
analysis = pd.concat([quanti, shapiro_test, t_test_paired, wilcoxon_test])

In [19]:
# write to xlsx file
analysis.to_excel("/media/guido/LACIE/Cingle_Guido/Project_band/Analysis_results/Analysis_Tables/analysis_diff_max_vs_firstfit_BH5P.xlsx",
                         sheet_name='diff_max_vs_firstfit')  